In [24]:
from dotenv import load_dotenv

load_dotenv()

True

In [25]:
responses = [{
  "user": "Kate",
  "answer": "During the massive floods last year, I worked with the local rescue team to provide emergency medical services. As a trained paramedic, I was responsible for administering first aid and ensuring the safety of evacuated residents.", 
  "ukraine":"I've regularly donated to charities that deliver medical supplies and food to Ukrainian refugees. Moving forward, I'd like to volunteer directly with these organizations to help distribute supplies on the ground.",
},{
  "user": "Alex",
  "ukraine":"As a blogger, I use my platform to share accurate information about the situation in Ukraine, combat misinformation, and promote fundraisers. I hope to collaborate with Ukrainian journalists to amplify their voices further.",
  "answer": "I collaborated with a group of local chefs to set up a community kitchen during the pandemic. My role was organizing the logistics, ensuring we had enough ingredients each day to prepare meals for hundreds of unemployed workers."
},{
  "user": "David",
  "ukraine":"I haven't had the chance to help yet, but I am planning to host a community art auction where local artists can donate their work, and proceeds will go directly to Ukrainian humanitarian aid.",
  "answer": "As the head of the neighborhood watch during the recent wildfires, I coordinated with the fire department and organized local volunteers to create firebreaks and assist in evacuation efforts."
},{
  "user": "Emily",
  "ukraine":"I joined a network that writes letters to soldiers and civilians in Ukraine to boost their morale. I also want to learn more about providing psychological first aid to better support those experiencing trauma.",
  "answer": "When our community was struck by a severe storm, I worked with the local church to convert their hall into a temporary shelter. I was in charge of bedding, providing comfort, and managing supplies for displaced families."
},{
  "user": "James",
  "ukraine":"I have donated to several NGOs that are providing on-the-ground support in Ukraine. In the future, I want to use my skills as a teacher to help educate displaced Ukrainian children online.",
  "answer": "I helped organize a tech drive to collect old laptops and smartphones for students when schools moved online. As a tech specialist, my role was to refurbish the devices to ensure they were ready for academic use."
},{
  "user": "Priya",
  "ukraine":"I've been part of a tech team developing secure communication tools for Ukrainian activists. Next, I'd like to help create educational apps to assist Ukrainian children continuing their studies during disruptions.",
  "answer": "During the economic downturn, I formed a partnership with local businesses to fundraise for those hardest hit. My role was to manage the crowdfunding campaigns and communicate with donors about where their contributions were going."
},{
  "user": "Tom",
  "ukraine":"My friends and I have been crafting blankets and warm clothing to send to Ukraine. I'm looking into how I can go there to help rebuild homes and community centers once it is safe to do so.",
  "answer": "I collaborated with the city council and other environmental activists to organize community clean-up events following the oil spill. My role involved planning the events, recruiting volunteers, and educating participants on safety procedures."
},{
  "user": "Zara",
  "ukraine":"I coordinated a local food drive to collect and send non-perishable items to Ukraine. I am now learning Ukrainian so I can offer more personalized support to Ukrainian refugees arriving in our community.",
  "answer": "After noticing the mental health struggles among elderly residents during isolation, I helped establish a virtual visitation program. As the program coordinator, I trained volunteers on how to use video call software and matched them with seniors in need of company."
},{
  "user": "Chris",
  "ukraine":"I started a petition to encourage our government to provide more aid and support to Ukraine. I am planning to get more involved in advocacy at a political level to ensure sustained support for Ukraine.",
  "answer": "When our local hospital was overwhelmed, I worked with a team of volunteers to provide non-medical support to healthcare staff. I took on the role of scheduling shifts and ensuring that all staff members got timely meals and rest breaks."
},{
  "user": "Will",
  "ukraine": "I organized a series of benefit concerts in my city to raise awareness and funds for Ukraine. We managed to gather a significant amount to support children affected by the war, and I plan to organize more such events.",
  "answer": "I led a team that set up an information dissemination service that helped non-English speaking residents understand the changing health regulations during the pandemic. My role was overseeing the translation and distribution of information in multiple languages."
}]

In [26]:
import spacy
nlp = spacy.load("en_core_web_sm")
def extract_entities(responses):
    for response in responses:
        doc = nlp(response)
        objects = []
        communities = []
        for token in doc:
            # Extracting Objects based on dependency parsing
            if token.dep_ in ['dobj', 'attr', 'pobj']:  # direct objects, attributes, object of preposition
                objects.append(token.text)


        print(f"Response: {response}")
        print("Extracted Objects:", list(set(objects)))
        print("Extracted Groups/Communities:", list(set(communities)))
        print()
def extract_entities(responses):
    data = []
    for response in responses:
        doc = nlp(response.get("answer", "") + " " + response.get("ukraine", ""))
        actions = set()
        roles = set()
        resources = set()
        communities = set()
        
        for token in doc:
            # Action extraction
            if token.pos_ == "VERB":
                actions.add(token.lemma_)
            # Role and resource extraction (simplified for demonstration)
            if token.pos_ == "NOUN":
                if "coordinator" in token.text:
                    roles.add(token.text)
                else:
                    resources.add(token.text)

            # Extracting Groups/Communities if mentioned explicitly
            if token.ent_type_ in ['NORP', 'ORG', 'GPE']:  # nationalities, organizations, geopolitical entities
                communities.add(token.text)
            # Additional heuristic: Check for noun chunks that might be groups
            for chunk in doc.noun_chunks:
                if 'community' in chunk.text or 'neighborhood' in chunk.text or 'families' in chunk.text:
                    communities.add(chunk.text)
        
        data.append({
            "user": response.get("user", "Anonymous"),
            "actions": list(actions),
            "roles": list(roles),
            "resources": list(resources),
            "communities": list(communities)
        })
    return data

# Extract data
extracted_data = extract_entities(responses)

In [27]:
import os
from neo4j import GraphDatabase, Driver, Transaction

# Create a driver instance
driver = GraphDatabase.driver(
    os.getenv("NEO4J_URI"),
    auth=(os.getenv("NEO4J_USERNAME"), os.getenv("NEO4J_PASSWORD"))
)
# Optionally, check if you want to verify connectivity at the start
driver.verify_connectivity()

# Clear the database
driver.session().run("MATCH (n) DETACH DELETE n")

In [28]:
# Function to insert data into Neo4j
def insert_data_into_neo4j(driver, extracted_data):
    with driver.session() as session:
        for entry in extracted_data:
            session.write_transaction(create_nodes_and_relationships, entry)

# Transaction function to create nodes and relationships
def create_nodes_and_relationships(tx, entry):
    # Create or find the user node
    tx.run("""
        MERGE (u:User {name: $name})
        SET u.answer = $answer, u.ukraine_help = $ukraine_help
        """, name=entry['user'], answer=entry['answer'], ukraine_help=entry['ukraine'])

    # Create action nodes and relationships
    for action in entry['actions']:
        tx.run("""
            MATCH (u:User {name: $name})
            MERGE (a:Action {name: $action})
            MERGE (u)-[:PERFORMS]->(a)
            """, name=entry['user'], action=action)

    # Create role nodes and relationships
    for role in entry['roles']:
        tx.run("""
            MATCH (u:User {name: $name})
            MERGE (r:Role {name: $role})
            MERGE (u)-[:ASSUMES]->(r)
            """, name=entry['user'], role=role)

    # Create resource nodes and relationships
    for resource in entry['resources']:
        tx.run("""
            MATCH (u:User {name: $name})
            MERGE (res:Resource {name: $resource})
            MERGE (u)-[:UTILIZES]->(res)
            """, name=entry['user'], resource=resource)

    # Create community nodes and relationships
    for community in entry['communities']:
        tx.run("""
            MATCH (u:User {name: $name})
            MERGE (c:Community {name: $community})
            MERGE (u)-[:BELONGS_TO]->(c)
            """, name=entry['user'], community=community)

In [29]:
# Insert data into Neo4j
insert_data_into_neo4j(driver, extracted_data)

# Close the Neo4j driver
driver.close()

/var/folders/yq/0zz796hs77l_70crv1b3x6z80000gn/T/ipykernel_79698/1101957894.py:5: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_nodes_and_relationships, entry)
